In [81]:
import math
import random
import numpy as np

from factor_module import check_prime

In [82]:
# Solves system of congruences x = a[i] mod n[i] by Chinese Remainder Theorem

def CRT(a, n):
    n_prod = math.prod(n)
    N = [n_prod // n_i for n_i in n]
    M = [pow(n_prod // n_i, -1, n_i) for n_i in n]

    return sum([a[i]*M[i]*N[i] for i in range(0, len(a))]) % n_prod


In [83]:
def PrimesRange(n):
    P = [2]
    for a in range(3, n):
        if check_prime(a):
            P.append(a)
    
    return P


def Factor(n):
    F = {}

    for p_i in PrimesRange(math.isqrt(n) + 1):
        k = 0
        while n % (p_i**(k+1)) == 0:
            k += 1

        if k > 0:
            F[p_i] = k
            n = n // (p_i**k)

    if n != 1:
        F[n] = 1

    return F

In [84]:
# Computes discrete log_a(b) modp by Silver–Pohlig–Hellman algorithm

def SPH(a, b, p):
    n = p - 1
    F = Factor(n)
    print(F)
    r_table = {}

    for p_i in F.keys():
        for j in range(0, p_i):
            t = pow(a, (n * j) // p_i, p)
            r_table[p_i, t] = j
    
    Y = []

    for p_i in F.keys():
        a_x_modn = 1
        x = 0
        for j in range(0, F[p_i]):
            t = pow(b * pow(a_x_modn, -1, p), n // (p_i ** (j+1)), p)
            x_j = r_table[p_i, t]
            x += (x_j * (p_i ** j))
            a_x_modn = (a_x_modn * pow(a, x_j * (p_i ** j))) % p
        
        Y.append(x)
    
    print(Y)
    P = [p_i**l_i for p_i, l_i in F.items()]
    print(P)
    return CRT(Y, P)



SPH(8, pow(8, 9, 997), 997)
print(pow(8, 9, 997) == pow(8, 341, 997))

{2: 2, 3: 1, 83: 1}
[1, 2, 9]
[4, 3, 83]
True
